Ohaaki Annual Seismic Report for Contact Energy
--
**Now July 1 to June 30**

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.transforms import offset_copy
from matplotlib import patheffects
import matplotlib.ticker as mticker

import numpy as np

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM
from cartopy.io.img_tiles import Stamen

import warnings; warnings.simplefilter('ignore')

%matplotlib inline

**Functions for scale bar**

This is a new scale_bar function from https://stackoverflow.com/questions/32333870/how-can-i-show-a-km-ruler-on-a-cartopy-matplotlib-plot as the last one started to give an error I couldn't understand

In [ ]:
def scale_bar(ax, length=None, location=(0.5, 0.05), linewidth=3):
    """
    ax is the axes to draw the scalebar on.
    length is the length of the scalebar in km.
    location is center of the scalebar in axis coordinates.
    (ie. 0.5 is the middle of the plot)
    linewidth is the thickness of the scalebar.
    """
    #Get the limits of the axis in lat long
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    #Make tmc horizontally centred on the middle of the map,
    #vertically at scale bar location
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    #Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(tmc)
    #Turn the specified scalebar location into coordinates in metres
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    #Calculate a scale bar length if none has been given
    #(Theres probably a more pythonic way of rounding the number but this works)
    if not length: 
        length = (x1 - x0) / 5000 #in km
        ndim = int(np.floor(np.log10(length))) #number of digits in number
        length = round(length, -ndim) #round to 1sf
        #Returns numbers starting with the list
        def scale_number(x):
            if str(x)[0] in ['1', '2', '5']: return int(x)        
            else: return scale_number(x - 10 ** ndim)
        length = scale_number(length) 

    #Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbx - length * 500, sbx + length * 500]
    #Plot the scalebar
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    #Plot the scalebar label
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

In [ ]:
# def utm_from_lon(lon):
#     """
#     utm_from_lon - UTM zone for a longitude

#     Not right for some polar regions (Norway, Svalbard, Antartica)

#     :param float lon: longitude
#     :return: UTM zone number
#     :rtype: int
#     """
#     return np.floor( ( lon + 180 ) / 6) + 1

# def scale_bar(ax, proj, length, location=(0.5, 0.05), linewidth=3,
#               units='km', m_per_unit=1000):
#     """

#     http://stackoverflow.com/a/35705477/1072212
#     ax is the axes to draw the scalebar on.
#     proj is the projection the axes are in
#     location is center of the scalebar in axis coordinates ie. 0.5 is the middle of the plot
#     length is the length of the scalebar in km.
#     linewidth is the thickness of the scalebar.
#     units is the name of the unit
#     m_per_unit is the number of meters in a unit
#     """
#     # find lat/lon center to find best UTM zone
#     x0, x1, y0, y1 = ax.get_extent(proj.as_geodetic())
#     # Projection in metres
#     utm = ccrs.UTM(utm_from_lon((x0+x1)/2))
#     # Get the extent of the plotted area in coordinates in metres
#     x0, x1, y0, y1 = ax.get_extent(utm)
#     # Turn the specified scalebar location into coordinates in metres
#     sbcx, sbcy = x0 + (x1 - x0) * location[0], y0 + (y1 - y0) * location[1]
#     # Generate the x coordinate for the ends of the scalebar
#     bar_xs = [sbcx - length * m_per_unit/2, sbcx + length * m_per_unit/2]
#     # buffer for scalebar
#     buffer = [patheffects.withStroke(linewidth=5, foreground="w")]
#     # Plot the scalebar with buffer
#     ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
#         linewidth=linewidth, path_effects=buffer)
#     # buffer for text
#     buffer = [patheffects.withStroke(linewidth=3, foreground="w")]
#     # Plot the scalebar label
#     t0 = ax.text(sbcx, sbcy, str(length) + ' ' + units, transform=utm,
#         horizontalalignment='center', verticalalignment='bottom',
#         path_effects=buffer, zorder=2)
#     left = x0+(x1-x0)*0.05
#     # Plot the N arrow
#     t1 = ax.text(left, sbcy, u'\u25B2\nN', transform=utm,
#         horizontalalignment='center', verticalalignment='bottom',
#         path_effects=buffer, zorder=2)
#     # Plot the scalebar without buffer, in case covered by text buffer
#     ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
#         linewidth=linewidth, zorder=3)
    
# # or to use m instead of km
# # scale_bar(ax, ccrs.Mercator(), 100000, m_per_unit=1, units='m')
# # or to use miles instead of km
# # scale_bar(ax, ccrs.Mercator(), 60, m_per_unit=1609.34, units='miles')

# #need specific font to show triangle for north arrow
# mpl.rc('font', family='DejaVu Sans')

In [ ]:
#box for search and map area
bbox = '176.2667,-38.5667,176.3500,-38.4833'
lonmin = 176.2667
lonmax = 176.3500
latmin = -38.5667
latmax = -38.4833

maxdepth = 20

**Edit these lines to set the current year**

In [ ]:
startyear = '2021-07-01T00:00:00'
endyear = '2022-07-01T00:00:00'

#as well as annual data, show number of events since 2010
startall = '2010-01-01T00:00:00'

**Quake search, get all events since startall**

In [ ]:
url = 'https://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&startdate='+startall+'&enddate='+endyear+'&maxdepth='+str(maxdepth)

In [ ]:
eqs = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime')

In [ ]:
eqs.head()

**Map for the reporting year**

In [ ]:
#select just events with an origintime (index) in the reporting year
eqsyr = eqs[(eqs.index>startyear)&(eqs.index<endyear)]

In [ ]:
len(eqsyr)

In [ ]:
eqsyr

In [ ]:
eqsyr.to_csv('ohaaki_eqs.csv')

In [ ]:
#plot map
fig = plt.figure(dpi=200)

#imagery
imagery = OSM()
# imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 15)

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(eqsyr['longitude'], eqsyr['latitude'], color='red', marker='o', s=eqsyr['magnitude']**2, transform=ccrs.Geodetic())

#power station text
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=0)

plt.text(176.2938, -38.5276, 'OP', verticalalignment='center', horizontalalignment='center', color='black', transform=text_transform, fontsize=4)

scale_bar(ax, 2)
plt.savefig('ohaaki_map.png', facecolor='white', bbox_inches='tight', dpi=400)

**Basic summary table of events for reporting year**

In [ ]:
table = eqsyr[['latitude', 'longitude', 'depth', 'magnitude']].copy()
table

In [ ]:
table['magnitude'].describe()

**Number of events each year, for a short table**

In [ ]:
eqs['publicid'].resample('A', label='right', closed='left').count()